<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/4_rag_recursive_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruções

* Estratégia
    * Usar segmentação por similaridade
    * Busca densa
    * Recursive Retriver

# Instalando pacotes

In [1]:
!pip install -q llama-index llama-index-llms-groq sentence-transformers llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# Importando pacotes

In [2]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [10]:
from llama_index.core import Settings, VectorStoreIndex, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.schema import TextNode, IndexNode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import FaithfulnessEvaluator, CorrectnessEvaluator, RelevancyEvaluator
from llama_index.core.response.notebook_utils import display_source_node, display_response
import json
import pandas as pd
from rich.pretty import pprint
from tqdm import tqdm
from IPython.display import Markdown, display

import nest_asyncio
nest_asyncio.apply()


# GitHub

In [4]:
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

Cloning into 'ia024-projeto-rag'...
remote: Enumerating objects: 6051, done.
remote: Counting objects: 100% (1056/1056), done.
remote: Compressing objects: 100% (378/378), done.
remote: Total 6051 (delta 665), reused 1045 (delta 658), pack-reused 4995
Receiving objects: 100% (6051/6051), 1.12 GiB | 31.05 MiB/s, done.
Resolving deltas: 100% (2054/2054), done.
Updating files: 100% (5788/5788), done.


In [5]:
DATA_DIR = '/content/ia024-projeto-rag'

In [6]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [7]:
def update_local_repo():
    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

In [ ]:
update_local_repo()

# RAG - Recursive Retriever

## Preparando dados

In [8]:
embed_model = HuggingFaceEmbedding(model_name="alfaneo/bertimbau-base-portuguese-sts")

Settings.embed_model = embed_model
Settings.llm         = Groq(model="llama3-70b-8192")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.39k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/438k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:

with open(f"{DATA_DIR}/2-dataset/estatuto.json", 'r') as f:
    documento = json.load(f)

documento_titulo = documento['titulo']
documento_texto  = documento['texto_compacto']

base_nodes = [
    TextNode(text=documento_texto, metadata={"doc_name": documento_titulo})
]

for parte in documento['partes']:
    metadata = {}
    metadata["doc_name"] = documento_titulo

    titulo_nome    = parte['titulo']
    titulo_artigos = parte['artigos']
    capitulos      = parte['capitulos']

    metadata["titulo"] = titulo_nome
    if titulo_artigos != "":
        node = TextNode(text=titulo_artigos, metadata=metadata)
        base_nodes.append(node)

    for capitulo in capitulos:
        cap_titulo  = capitulo['titulo']
        cap_artigos = capitulo['artigos']

        metadata["capitulo"] = cap_titulo
        node = TextNode(text=cap_artigos, metadata=metadata)
        base_nodes.append(node)

for i, node in enumerate(base_nodes):
    node.id_ = f"node_{i}"

len(base_nodes)

18

## Construindo Retriever e Query Engine

In [13]:
# Set Up Recursive Retriever
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [SentenceSplitter(chunk_size=c, chunk_overlap=20) for c in sub_chunk_sizes]

all_nodes = []
for base_node in base_nodes:

    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes]
        all_nodes.extend(sub_inodes)

    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

all_nodes_dict = {n.node_id: n for n in all_nodes}


vector_index = VectorStoreIndex(all_nodes)


In [27]:
vector_retriever = vector_index.as_retriever(similarity_top_k=2)

retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=all_nodes_dict,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    verbose=True
)

In [28]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} " f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [29]:
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template**Text:** 

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: response_synthesizer:refine_template**Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [30]:
new_summary_tmpl_str = ("""
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer such as Document, Title, Chapter, or Article when provided.
Answer in the same language as the query.
Query: {query_str}
Source:
Answer:
"""
)

new_refine_tmpl_str = ("""
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Provide the source of the answer such as Document, Title, Chapter, or Article when provided.
Answer in the same language as the query.
Refined Answer:

"""
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)
new_refine_tmpl_str = PromptTemplate(new_refine_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": new_summary_tmpl
     , "response_synthesizer:refine_template" : new_refine_tmpl_str}
)

display_prompt_dict(query_engine.get_prompts())


**Prompt Key**: response_synthesizer:text_qa_template**Text:** 


Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer such as Document, Title, Chapter, or Article when provided.
Answer in the same language as the query.
Query: {query_str}
Source:
Answer:



**Prompt Key**: response_synthesizer:refine_template**Text:** 


The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Provide the source of the answer such as Document, Title, Chapter, or Article when provided.
Answer in the same language as the query.
Refined Answer: 




In [31]:
#query = "Qual o nome do IFSP?"
#query = "Quais são os campi do IFSP? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quantos campi o IFSP tem? Informe onde você encontrou essa informação"
#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
query = "O Art 4º do estatuto do IFSP é sobre o que?"

response = query_engine.query(query)
display_response(response)

Retrieving with query id None: O Art 4º do estatuto do IFSP é sobre o que?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: O Art 4º do estatuto do IFSP é sobre o que?
Retrieved node with id, entering: node_1
Retrieving with query id node_1: O Art 4º do estatuto do IFSP é sobre o que?


**`Final Response:`** The new context provided does not add any information that would refine the original answer. The original answer is still the most accurate response to the query.

Refined Answer: O Art 4º do estatuto do IFSP é sobre os princípios norteadores do IFSP.

Source: Estatuto, Capítulo II, Art 4º

# Avaliando conjunto de perguntas e respostas

In [32]:
pairs = [{
            "question": "Qual o nome do IFSP?",
            "answer": "Instituto Federal de Educação, Ciência e Tecnologia de São Paulo.  Referência: Estatuto, Título I, Capítulo I, Art. 1º"
         }
         ,{
            "question": "Quais são os campi do IFSP?",
            "answer": """II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.
IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã,
Sertãozinho.
V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de
Janeiro, Guarulhos.
VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º,
Jardim Itália, São João da Boa Vista.
VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá,
Caraguatatuba.
VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho,
770, Penha, Bragança Paulista.
IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.
X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição,
São Roque.
XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119,
São Carlos.
XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila
Abernéssia, Campos do Jordão.
XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.
XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano,
Suzano.
XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais,
Campinas;
XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial,
Catanduva.
XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim
Europa, Avaré.
XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza
Pinheiro, Araraquara.
XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen,
Itapetininga.
XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II,
Birigui.
XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.
XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá,
Registro.
XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50,
Jardim Tropical, Presidente Epitácio.
XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro
Santa Rosa, Piracicaba.
XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896,
Hortolândia.
XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos
Pássaros, Boituva.
XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João
Batista, Capivari.
XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.
XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145,
Jardim Diamante, São José dos Campos.

Referência: Estatuto, Título I, Capítulo I, Art. 1º, §2º
"""
         }
         ,{
            "question": "Quantos campi o IFSP tem? Informe onde você encontrou essa informação",
            "answer": "O IFSP tem 28 campi. Referência: Estatuto, Capítulo I, Art. 1º, §2º"
         }
        ]

#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"

In [33]:
faith_evaluator     = FaithfulnessEvaluator()
relevancy_evaluator = RelevancyEvaluator()
correct_evaluator   = CorrectnessEvaluator()
data = []

for pair in tqdm(pairs):
    result       = {}
    query        = pair['question']
    ground_truth = pair['answer']

    result['question']     = query
    result['ground_truth'] = ground_truth

    # Fazendo consulta
    response = query_engine.query(query)
    result['answer'] = response.response

    # Avaliando Faithfulness
    eval_result = faith_evaluator.evaluate_response(response=response)
    result['faithfulness'] = eval_result.score

    # Avaliando Relevancy
    eval_result = relevancy_evaluator.evaluate_response(query=query, response=response)
    result['relevancy'] = eval_result.score

    # Avaliando Correctness
    eval_result = correct_evaluator.evaluate(
        query=query,
        response=response.response,
        reference=ground_truth,
    )
    result['correctness'] = eval_result.score
    result['correcteness_feedback'] = eval_result.feedback

    data.append(result)


df = pd.DataFrame(data)
df

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving with query id None: Qual o nome do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Qual o nome do IFSP?
Retrieved node with id, entering: node_1
Retrieving with query id node_1: Qual o nome do IFSP?


 33%|███▎      | 1/3 [06:04<12:08, 364.42s/it]

Retrieving with query id None: Quais são os campi do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quais são os campi do IFSP?


 67%|██████▋   | 2/3 [15:52<08:16, 496.13s/it]

Retrieving with query id None: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quantos campi o IFSP tem? Informe onde você encontrou essa informação


100%|██████████| 3/3 [22:01<00:00, 440.50s/it]


,question,ground_truth,answer,faithfulness,relevancy,correctness,correcteness_feedback
0,Qual o nome do IFSP?,"Instituto Federal de Educação, Ciência e Tecno...",The new context does not add any information t...,1.0,1.0,5.0,The generated answer is fully correct and rele...
1,Quais são os campi do IFSP?,"II. Campus São Paulo, situado na Rua Pedro Vic...",The new context does not provide any additiona...,1.0,1.0,5.0,The generated answer is fully correct and rele...
2,Quantos campi o IFSP tem? Informe onde você en...,"O IFSP tem 28 campi. Referência: Estatuto, Cap...",The refined answer remains the same: O IFSP te...,1.0,0.0,3.0,The generated answer is relevant to the user q...
